In [56]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches
import os
import glob
import numpy as np
from typing import Iterable
import pickle
from tqdm.notebook import tqdm
import tensorflow as tf
from scipy.fft import fft, ifft
from statsmodels.tsa.stattools import acf, pacf
from sklearn.decomposition import PCA

import keras
import keras_tuner as kt
from kerastuner import RandomSearch
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [57]:
# load a single shot using pandas parquet file reader 
def load_shot(shotnum: int) -> pd.DataFrame:
    shot = pd.read_parquet("../data/DISR_data/JET_DATAno" + str(shotnum) + ".parquet")
    return shot

df_shot = load_shot(96885)
df_shot['IPE'] = df_shot['IPLA'] - df_shot['IPref']
display(df_shot)

,time,IPLA,IPref,ECEcore,SSXcore,LI,Q95,ZMAG,Vloop,IPE
0,0.022000,-53160.492188,0.022461,1141.500610,0.002289,0.661752,10.408838,0.020125,-17.961126,-53160.515625
1,0.023000,-56066.082031,0.022461,1141.500610,0.001373,0.661752,10.408838,0.020125,-17.642231,-56066.105469
2,0.024000,-59827.996094,0.022461,1141.500610,0.000916,0.661752,10.408838,0.020125,-18.239368,-59828.019531
3,0.025000,-63139.011719,0.022461,1141.500610,0.001679,0.661752,10.408838,0.020125,-18.200048,-63139.035156
4,0.026000,-66272.710938,0.022461,1141.500610,0.001221,0.661752,10.408838,0.020125,-17.801512,-66272.734375
...,...,...,...,...,...,...,...,...,...,...
16880,16.902000,-100682.593750,0.022461,1224.449097,0.002441,1.155780,3.558453,0.329814,-55.337883,-100682.617188
16881,16.903000,-96515.125000,0.022461,1224.449097,0.001831,1.155780,3.558453,0.329814,-49.116920,-96515.148438
16882,16.903999,-85514.335938,0.022461,1224.449097,0.002441,1.155780,3.558453,0.329814,-43.695698,-85514.359375
16883,16.905001,-70803.953125,0.022461,1224.449097,0.002899,1.155780,3.558453,0.329814,-37.757305,-70803.976562


In [58]:
model = tf.keras.models.load_model("../models/nn_window.keras")

window = 0

# Add 'Instability' and 'Window' attributes to the DataFrame
df_shot['Instability'] = 0
df_shot['Window'] = window
df_shot['Label'] = -1

# Define the window size and the number of samples with the same label required
window_size = 20
window_cursor_one = 0
window_cursor_two = window_size - 1
label_count = 15

# Iterate over the samples in the DataFrame
for i in tqdm(range(len(df_shot)), desc='Processing samples'):
    # Get the current sample
    sample = df_shot.iloc[i]
    
    # Perform the prediction using the Keras model
    prediction = model.predict(np.array([sample]))[0]
    label = np.argmax(prediction)

    df_shot.loc[i, 'Label'] = label

    max_label = df_shot['Label'].iloc[window_cursor_one : window_cursor_two].value_counts().max()
    
    # Check if there are enough samples with the same label in the window
    if max_label >= label_count and max_label != -1:
        window += 1
        # Set the window attribute for the samples inside the window
        df_shot.loc[window_cursor_one : window_cursor_two, 'Window'] = window

        # Set the label attribute for the samples inside the window
        df_shot.loc[window_cursor_one : window_cursor_two, 'Label'] = max_label
        
        # Move the window to the next 20 samples
        window_cursor_one += window_size
        window_cursor_two += window_size

        # Check if the label is 1
        if label == 1:
            # Increase the instability by 1 for all following samples
            df_shot.loc[i + 1:, 'Instability'] += 1

    elif i == window_cursor_two:
        # Move the window to the next samples
        window_cursor_one += 1
        window_cursor_two += 1

# Display the updated DataFrame
display(df_shot)

Processing samples:   0%|          | 0/16885 [00:00<?, ?it/s]

1/1 [==============================] - 0s 22ms/step


In [52]:
windows = df_shot[df_shot['Label'] == 1]['Window'].unique()

display(df_shot[df_shot['Window'] == windows[20]])

,time,IPLA,IPref,ECEcore,SSXcore,LI,Q95,ZMAG,Vloop,IPE,Instability,Window,Label
1160,1.182,-838739.8125,-825310.2500,1253.171143,0.007630,1.131199,6.097872,0.203220,-1.912041,-13429.5625,66,59,1
1161,1.183,-840914.2500,-825310.2500,1257.291016,0.007477,1.131199,6.097872,0.203220,-2.031336,-15604.0000,66,59,1
1162,1.184,-839739.1875,-827977.1875,1256.148682,0.006867,1.131199,6.097872,0.203220,-2.350231,-11762.0000,66,59,1
1163,1.185,-840095.4375,-827977.1875,1253.816650,0.006561,1.131199,6.097872,0.203220,-1.275249,-12118.2500,66,59,1
1164,1.186,-841241.4375,-827977.1875,1256.504395,0.006867,1.131199,6.097872,0.203220,-1.672453,-13264.2500,66,59,1
1165,1.187,-840995.4375,-827977.1875,1261.497070,0.006561,1.131199,6.097872,0.203220,-2.150630,-13018.2500,66,59,1
1166,1.188,-842178.8750,-827977.1875,1251.570435,0.007630,1.131199,6.097872,0.203220,-1.870722,-14201.6875,66,59,1
1167,1.189,-843018.1875,-827977.1875,1256.684814,0.007630,1.143787,6.264678,0.206482,-1.872054,-15041.0000,66,59,1
1168,1.190,-843763.8750,-827977.1875,1252.839478,0.007324,1.143787,6.264678,0.206482,-1.871388,-15786.6875,66,59,1
1169,1.191,-844171.2500,-827977.1875,1258.702881,0.007172,1.143787,6.264678,0.206482,-1.871388,-16194.0625,66,59,1


In [54]:
# Filter out rows where 'Window' is 0
df_shot_filtered = df_shot[df_shot['Window'] != 0]

# Remove the 'Label' column
df_shot_modified = df_shot_filtered.drop(columns=['Label'])

# Save the DataFrame as a csv file
df_shot_modified.to_csv("../data/pipeline.csv", index=False)

(844, 20, 11)